In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
#loading the data with $eta = 0.25$, $omega_c = 0.5$ with $s \in [0, 0.5] if the spectral density is sub-Ohmic or 
#$s \in [1.5, 4]$ is the spectral density is super-Ohmic
Xtrainseparated = np.loadtxt('../Data/Xtrainx_fixedηandω_separated.csv', delimiter=',') #load data for sigmax
Ytrainseparated = np.loadtxt('../Data/Ytrain_fixedηandω_separated.csv', delimiter=',') #load labels

In [ ]:
#function to take X and Y and return a training, test and validation set containing the Fourier coefficients, along
#with the corresponding labels. The last 3 columns of Y contain the values of $'\eta'$, $\omega_c$ and $s$ so we 
#only use the first three columns. Likewise, at t=0 all of the signals are equal, so we omit the first timestep.
def fouriertrainvaltest(X, Y):
    
    #Generating a training set with 4800 trajectories, and a validation and test set each with 2400 trajectories.
    Xtrain = X[0:4800, 1:401]
    Xval = X[4800:7200, 1:401]
    Xtest = X[7200:9600, 1:401]

    Ytrain = Y[0:4800, 0:3]
    Yval = Y[4800:7200, 0:3]
    Ytest = Y[7200:9600, 0:3]
    
    #calculating the Fourier coefficients
    XtrainF = np.fft.fft(Xtrain)
    XvalF = np.fft.fft(Xval)
    XtestF = np.fft.fft(Xtest)
    
    #Splitting the coefficients up into their real and imaginary components
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))
    
    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag
    
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag
        
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag
    
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
#an array to store the training accuracy at each standard deviation
trainingaccuracyseparated=np.zeros(10)

#an array to store the test accuracy at each standard deviation
testaccuracyseparated = np.zeros(10)

In [ ]:
for i in range(10):
    
    #Adding artificial noise to the dataset
    Xtrainseparatederrorarr = np.zeros_like(Xtrainseparated)
    
    for j in range(Xtrainseparated.shape[0]):
        for k in range(Xtrainseparated.shape[1]):
            Xtrainseparatederrorarr[j,k] = Xtrainseparated[j,k] + np.random.normal(0, 0.1+i*0.1)
    
    #Generating the training, validation and test set
    xtrainseparatederror, xvalseparatederror, xtestseparatederror, ytrainseparatederror, yvalseparatederror, ytestseparatederror = fouriertrainvaltest(Xtrainseparatederrorarr, Ylabelsseparated)
    
    #Defining the model
    modelseparated = tf.keras.Sequential()
    modelseparated.add(tf.keras.layers.Dense(250, input_dim = (np.shape(xtrainseparatederror)[1]), activation='sigmoid'))
    modelseparated.add(tf.keras.layers.Dense(80 , activation = 'sigmoid'))
    modelseparated.add(tf.keras.layers.Dense(3, activation = 'softmax'))

    #Setting the optimiser equal to the Adam optimiser with learning rate = 0.0001
    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001)

    #Compliling the model
    modelseparated.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

    #Training the model
    historyseparated = modelseparated.fit(xtrainseparatederror, ytrainseparatederror, epochs = 1000, validation_data = (xvalseparatederror, yvalseparatederror), batch_size = np.shape(xtrainseparatederror)[0], verbose=0)
    
    #Finding the training accuracy
    trainingaccuracyseparated[i] = modelseparated.evaluate(xtrainseparatederror, ytrainseparatederror)[1]
    
    #Finding the test accuracy
    testaccuracyseparated[i] = modelseparated.evaluate(xtestseparatederror, ytestseparatederror)[1]